# 套件安裝

In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests

# 設定初始化

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 國立清華大學 中國文學系 清華中文學報
https://cl.site.nthu.edu.tw/p/403-1401-3772-1.php?Lang=zh-tw  
'''


'''
匯入套件
'''
# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
prefix_url = 'https://cl.site.nthu.edu.tw/'
url = prefix_url + 'p/403-1401-3772-1.php?Lang=zh-tw'

# 指定 sheet name
sheetName = 'cl_site_nthu'

# 指定 excel 檔名
excelFileName = 'cl_site_nthu.xlsx'

# 指定 json 檔名
jsonFileName = f'{sheetName}.json'

# 建立儲存圖片、影片的資料夾
folderPath = f'./{sheetName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 判斷 excel 檔案是否存在，不存在就新增
filePath = folderPath + '/' + excelFileName
if not os.path.exists(filePath):
    workbook = Workbook() # 動態新增檔案
    worksheet = workbook.create_sheet(sheetName, 0) # 建立並取得 active sheet
else:
    workbook = load_workbook(filename = filePath)
    worksheet = workbook[sheetName] # 取得 active sheet

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": folderPath
})
    
# excel 標題
worksheet['A1'] = "流水號"
worksheet['B1'] = "期刊名稱"
worksheet['C1'] = '網頁連結'
worksheet['D1'] = "出版日期"
worksheet['E1'] = "論文名稱"
worksheet['F1'] = "作者名稱"
worksheet['G1'] = "論文連結"
worksheet['H1'] = "論文連結_curl"

# 放置爬取的資料
listData = []

# 自訂函式 (網路爬蟲執行流程)

In [ ]:
'''
函式
'''
# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )
    
# 走訪來源網頁
def visit():
    global driver
    driver.get(url) #進入來源網頁
    try:
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.row.listBS.boxSD a[href]')
            )
        )
    except TimeoutException as e:
        print('等待逾時: visit')
        
# 滾動頁面
def scroll():
    global driver
    
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 1
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        '''
        或是每次只滾動一點距離，
        以免有些網站會在移動長距離後，
        將先前移動當中的元素隱藏

        例如將上方的 script 改成:
        offset += 600
        '''

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        
        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(1)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        );
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1
    
# 剖析內容
def parse():
    global driver
    global listData
    try:
        # 流水號
        sn = 1
        
        # 暫存頁面資訊
        listTmp = []
        
        # 取得主要連結
        for a in driver.find_elements(By.CSS_SELECTOR, 'div.row.listBS.boxSD a[href]'):
            # 網頁連結
            strMainLink = a.get_attribute('href')
            
            # 期刊名稱 與 出版日期
            strMainName = strMainDate = ''
            regexMain = r'.+（(\d{4}\.\d{1,2})）'
            matchMain = re.search(regexMain, a.get_attribute('innerText'))
            if matchMain != None:
                strMainName = matchMain[0] # 期刊名稱
                strMainDate = matchMain[1] # 出版日期
                
            listTmp.append({
                '期刊名稱': strMainName,
                '網頁連結': strMainLink,
                '出版日期': strMainDate,
            })
        
        # 取得資料
        for d in listTmp:
            # 取得內頁資料
            driver.get(d['網頁連結'])
            
            # 等待目標元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div.meditor table tbody tr')
                )
            )
            
            # 取得期刊篇名與作者名稱
            tr_elms = driver.find_elements(By.CSS_SELECTOR, 'div.meditor table tbody tr')
            for tr in tr_elms:                
                # pdf 連結與篇名、作者
                strPdfLink = strJournalTitle = strAuthor = ''
                
                # 若 td 數量等於 2，代表是期刊連結的那一列
                if len(tr.find_elements(By.CSS_SELECTOR, 'td')) == 2:
                    # 取得 td
                    td_elms = tr.find_elements(By.CSS_SELECTOR, 'div.meditor table tbody tr td')
                    
                    # 有 a 才取得資料 (有些 tr 當中的 td 裡面，沒有 a)
                    if len(td_elms[0].find_elements(By.CSS_SELECTOR, 'a')) > 0:
                        a_elm = td_elms[0].find_element(By.CSS_SELECTOR, 'a[href]')

                        # pdf 連結
                        strPdfLink = a_elm.get_attribute('href')

                        # 篇名
                        strJournalTitle = td_elms[0].get_attribute('innerText')
                        strJournalTitle = strJournalTitle.strip()
                        strJournalTitle = re.sub(r"\n", "", strJournalTitle)
                        
                        # 作者
                        strAuthor = td_elms[1].get_attribute('innerText')
                        strAuthor = strAuthor.strip()

                        # 整理資料
                        listData.append({
                            '流水號': sn,
                            '期刊名稱': d['期刊名稱'],
                            '網頁連結': d['網頁連結'],
                            '出版日期': d['出版日期'],
                            '論文名稱': strJournalTitle,
                            '作者名稱': strAuthor,
                            '論文連結': strPdfLink,
                            '論文連結_curl': strPdfLink
                        })

                        # 流水號遞增
                        sn += 1
    except TimeoutException as e:
        print('等待逾時: parse')
        
# 關閉瀏覽器
def close():
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )

# 儲存成 excel
def saveExcel():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:
        # 從 excel 列號 2 開始寫入資料
        row_num = 2
        
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
        # 逐列寫入
        for myDict in listJson:
            worksheet['A' + str(row_num)] = myDict["流水號"]
            worksheet['B' + str(row_num)] = myDict["期刊名稱"]
            worksheet['C' + str(row_num)] = myDict["網頁連結"]
            worksheet['D' + str(row_num)] = myDict["出版日期"]
            worksheet['E' + str(row_num)] = myDict["論文名稱"]
            worksheet['F' + str(row_num)] = myDict["作者名稱"]
            worksheet['G' + str(row_num)] = myDict["論文連結"]
            worksheet['H' + str(row_num)] = myDict["論文連結_curl"]
            row_num += 1
    
    # 儲存 workbook
    workbook.save(filePath)

    # 關閉 workbook
    workbook.close()
    
# 下載
def download():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
        for myDict in listJson:
            # 等待
            sleep(randint(1,3))
            
            # 下載 pdf
            cmd = ['curl', '-L', myDict["論文連結_curl"], '-o', f'{folderPath}/sn_{myDict["流水號"]}.pdf']
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            #output = result.stdout
            #pprint(output)
            print(f'{folderPath}/sn_{myDict["流水號"]}.pdf')

# 以下函式，請各別依情況分別、陸續執行

In [ ]:
# 初始化 Web Driver
init()

In [ ]:
# 走訪來源網頁
visit()

In [ ]:
# 滾動頁面
scroll()

In [ ]:
# 剖析內容
parse()

In [ ]:
# 關閉瀏覽器
close()

In [ ]:
# 儲存成 json
saveJson()

In [ ]:
# 儲存成 excel
saveExcel()

In [ ]:
# 下載
download()